## Load Libraries and Data Sets

In [1]:
import numpy as np
import pandas as pd
from lightgbm import LGBMClassifier
from skopt import BayesSearchCV
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, KFold, train_test_split, cross_val_score
from sklearn.metrics import roc_auc_score, roc_curve, plot_roc_curve, 
from sklearn.decomposition import PCA
from sklearn import tree
from sklearn import linear_model
from sklearn import neighbors, datasets
from sklearn.svm import SVC
from tqdm import tqdm
from sklearn import preprocessing

In [5]:
# data from the feature engineering notebook
train_feat = pd.read_pickle("train_feat3.pkl")
train_lab = pd.read_pickle("train_lab3.pkl")
test_feat = pd.read_pickle("test_feat3.pkl")

In [21]:
# Sample some of the data for nested cross validation
X_non, X_sample, y_non, y_sample = train_test_split(train_feat, train_lab, test_size = 0.05,random_state = 6420)

In [23]:
sum(y_sample)/len(y_sample)

0.07986472424557753

## Set up Nested Cross Validation

In [24]:
# training testing split
X_train, X_test, Y_train, Y_test = train_test_split(X_sample, y_sample, test_size = 0.2, random_state=6420)

In [25]:
# standardize features
std_scale = preprocessing.StandardScaler().fit(X_train)
X_train_std = std_scale.transform(X_train)
X_test_std = std_scale.transform(X_test)

In [26]:
# use PCA to cut down number of features to try in NCV
pca = PCA()
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)
pd.DataFrame(pca.explained_variance_ratio_)[:20].sum()

0    0.467999
dtype: float64

In [27]:
pca_std = PCA()
X_train_std_pca = pca_std.fit_transform(X_train_std)
X_test_std_pca = pca_std.transform(X_test_std)
pd.DataFrame(pca_std.explained_variance_ratio_)[:20].sum()

0    0.274606
dtype: float64

In [28]:
# choosing top 20 PCA transformed, standardized features for NCV
pca_std = PCA(20)
X_train_std_pca = pca_std.fit_transform(X_train_std)
X_test_std_pca = pca_std.transform(X_test_std)

In [29]:
# initiate models
dt = tree.DecisionTreeClassifier(random_state=6420)
lr = linear_model.LogisticRegression(random_state = 6420)
knn = neighbors.KNeighborsClassifier()
svc = SVC()
lgbm = LGBMClassifier()

In [43]:
# set up grids
dt_grid = {'criterion': ['gini', 'entropy'], 'max_depth': range(1, 55, 10)}
knn_grid = {'n_neighbors': range(1, 55, 10), 'weights': ["uniform", "distance"]}
lr_grid = {'C': [0.1, 1, 10, 100, 1000]}
svc_grid = {'C': [0.1, 1, 10, 100, 1000],  
            'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
            'kernel': ['rbf']}
lgbm_grid = {'boosting_type': ['gbdt', 'dart', 'goss'],
             'num_leaves': range(50, 500, 50),
             'min_child_samples': range(100, 1000, 250),
             'scale_pos_weight': range(1, 15, 3),
             'n_estimators': [100, 200, 500]}

In [44]:
NUM_TRIALS = 5
dt_non_nested_scores = np.zeros(NUM_TRIALS)
dt_nested_scores = np.zeros(NUM_TRIALS)
knn_non_nested_scores = np.zeros(NUM_TRIALS)
knn_nested_scores = np.zeros(NUM_TRIALS)
lr_non_nested_scores = np.zeros(NUM_TRIALS)
lr_nested_scores = np.zeros(NUM_TRIALS)
svc_non_nested_scores = np.zeros(NUM_TRIALS)
svc_nested_scores = np.zeros(NUM_TRIALS)
lgbm_non_nested_scores = np.zeros(NUM_TRIALS)
lgbm_nested_scores = np.zeros(NUM_TRIALS)

In [45]:
# ncv - auc as metric
for i in tqdm(range(NUM_TRIALS)):
    inner_cv = KFold(n_splits=3, shuffle=True, random_state=i)
    outer_cv = KFold(n_splits=3, shuffle=True, random_state=i)
    scoring = 'roc_auc'
    
    # Non_nested parameter search and scoring
    dt_clf = RandomizedSearchCV(estimator=dt, param_distributions=dt_grid, scoring=scoring, cv=inner_cv, n_iter=5)
    knn_clf = RandomizedSearchCV(estimator=knn, param_distributions=knn_grid, scoring=scoring, cv=inner_cv, n_iter=5)
    lr_clf = RandomizedSearchCV(estimator=lr, param_distributions=lr_grid, scoring=scoring, cv=inner_cv, n_iter=5)
    svc_clf = RandomizedSearchCV(estimator=svc, param_distributions=svc_grid, scoring=scoring, cv=inner_cv, n_iter=5)
    lgbm_clf = RandomizedSearchCV(estimator=lgbm, param_distributions=lgbm_grid, scoring=scoring, cv=inner_cv, n_iter=5)
    
    # Fit each model
    dt_clf.fit(X_train_std_pca, Y_train)
    knn_clf.fit(X_train_std_pca, Y_train)
    lr_clf.fit(X_train_std_pca, Y_train)
    svc_clf.fit(X_train_std_pca, Y_train)
    lgbm_clf.fit(X_train_std_pca, Y_train)
    
    dt_non_nested_scores[i] = dt_clf.best_score_
    knn_non_nested_scores[i] = knn_clf.best_score_
    lr_non_nested_scores[i] = lr_clf.best_score_
    svc_non_nested_scores[i] = svc_clf.best_score_
    lgbm_non_nested_scores[i] = lgbm_clf.best_score_
    
    print("Trial", i,
          "dt", dt_clf.best_params_, dt_non_nested_scores[i],
          "knn", knn_clf.best_params_, knn_non_nested_scores[i],
          "lr", lr_clf.best_params_, lr_non_nested_scores[i],
          "svc", svc_clf.best_params_, svc_non_nested_scores[i],
          "lgbm", lgbm_clf.best_params_, lgbm_non_nested_scores[i]
         )
    
    # Nested CV with parameter optimization
    dt_nested_score = cross_val_score(estimator=dt_clf, X=X_train_std_pca, y=Y_train, cv=outer_cv)
    dt_nested_scores[i] = dt_nested_score.mean()
    knn_nested_score = cross_val_score(estimator=knn_clf, X=X_train_std_pca, y=Y_train, cv=outer_cv)
    knn_nested_scores[i] = knn_nested_score.mean()
    lr_nested_score = cross_val_score(estimator=lr_clf, X=X_train_std_pca, y=Y_train, cv=outer_cv)
    lr_nested_scores[i] = lr_nested_score.mean()
    svc_nested_score = cross_val_score(estimator=svc_clf, X=X_train_std_pca, y=Y_train, cv=outer_cv)
    svc_nested_scores[i] = svc_nested_score.mean()
    lgbm_nested_score = cross_val_score(estimator=lgbm_clf, X=X_train_std_pca, y=Y_train, cv=outer_cv)
    lgbm_nested_scores[i] = lgbm_nested_score.mean()
print(dt_nested_scores,
      knn_nested_scores,
      lr_nested_scores,
      svc_nested_scores,
      lgbm_nested_scores
     )








  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

Trial 0 dt {'max_depth': 11, 'criterion': 'gini'} 0.5615122579555772 knn {'weights': 'uniform', 'n_neighbors': 51} 0.6265206712268446 lr {'C': 0.1} 0.6823584494918533 svc {'kernel': 'rbf', 'gamma': 0.01, 'C': 100} 0.5607566104194437 lgbm {'scale_pos_weight': 1, 'num_leaves': 100, 'n_estimators': 100, 'min_child_samples': 350, 'boosting_type': 'gbdt'} 0.6545804725170946








 20%|████████████████▌                                                                  | 1/5 [03:41<14:45, 221.48s/it]

Trial 1 dt {'max_depth': 1, 'criterion': 'entropy'} 0.5660211897136161 knn {'weights': 'distance', 'n_neighbors': 31} 0.5973409235886357 lr {'C': 0.1} 0.6811871523126315 svc {'kernel': 'rbf', 'gamma': 0.01, 'C': 100} 0.5573337731213864 lgbm {'scale_pos_weight': 1, 'num_leaves': 200, 'n_estimators': 100, 'min_child_samples': 850, 'boosting_type': 'dart'} 0.6709587808101344








 40%|█████████████████████████████████▏                                                 | 2/5 [09:48<13:15, 265.08s/it]

...

Trial 3 dt {'max_depth': 11, 'criterion': 'gini'} 0.5564065843847991 knn {'weights': 'distance', 'n_neighbors': 51} 0.62191804482474 lr {'C': 0.1} 0.68185172616699 svc {'kernel': 'rbf', 'gamma': 0.001, 'C': 10} 0.5702614319127596 lgbm {'scale_pos_weight': 4, 'num_leaves': 300, 'n_estimators': 100, 'min_child_samples': 850, 'boosting_type': 'gbdt'} 0.6630210817031776








 80%|██████████████████████████████████████████████████████████████████▍                | 4/5 [18:18<04:18, 258.19s/it]

Trial 4 dt {'max_depth': 11, 'criterion': 'gini'} 0.5716502159124157 knn {'weights': 'distance', 'n_neighbors': 51} 0.6266426737011406 lr {'C': 0.1} 0.6833089923719392 svc {'kernel': 'rbf', 'gamma': 0.01, 'C': 10} 0.5624774630407371 lgbm {'scale_pos_weight': 1, 'num_leaves': 100, 'n_estimators': 100, 'min_child_samples': 350, 'boosting_type': 'goss'} 0.670031859815252








100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [22:21<00:00, 268.25s/it]

[0.55310415 0.56602119 0.5394324  0.56714977 0.56853471] [0.62207463 0.61355956 0.6108067  0.6145908  0.62381038] [0.68235845 0.68118715 0.68355812 0.68185173 0.68330899] [0.55676974 0.55058374 0.56562047 0.54872907 0.54474883] [0.67025799 0.6612907  0.66985163 0.65230957 0.66459423]


In [46]:
# ncv - recall as metric
for i in tqdm(range(NUM_TRIALS)):
    inner_cv = KFold(n_splits=3, shuffle=True, random_state=i)
    outer_cv = KFold(n_splits=3, shuffle=True, random_state=i)
    scoring = 'recall'
    
    # Non_nested parameter search and scoring
    dt_clf = RandomizedSearchCV(estimator=dt, param_distributions=dt_grid, scoring=scoring, cv=inner_cv, n_iter=5)
    knn_clf = RandomizedSearchCV(estimator=knn, param_distributions=knn_grid, scoring=scoring, cv=inner_cv, n_iter=5)
    lr_clf = RandomizedSearchCV(estimator=lr, param_distributions=lr_grid, scoring=scoring, cv=inner_cv, n_iter=5)
    svc_clf = RandomizedSearchCV(estimator=svc, param_distributions=svc_grid, scoring=scoring, cv=inner_cv, n_iter=5)
    lgbm_clf = RandomizedSearchCV(estimator=lgbm, param_distributions=lgbm_grid, scoring=scoring, cv=inner_cv, n_iter=5)
    
    # Fit each model
    dt_clf.fit(X_train_std_pca, Y_train)
    knn_clf.fit(X_train_std_pca, Y_train)
    lr_clf.fit(X_train_std_pca, Y_train)
    svc_clf.fit(X_train_std_pca, Y_train)
    lgbm_clf.fit(X_train_std_pca, Y_train)
    
    dt_non_nested_scores[i] = dt_clf.best_score_
    knn_non_nested_scores[i] = knn_clf.best_score_
    lr_non_nested_scores[i] = lr_clf.best_score_
    svc_non_nested_scores[i] = svc_clf.best_score_
    lgbm_non_nested_scores[i] = lgbm_clf.best_score_
    
    print("Trial", i,
          "dt", dt_clf.best_params_, dt_non_nested_scores[i],
          "knn", knn_clf.best_params_, knn_non_nested_scores[i],
          "lr", lr_clf.best_params_, lr_non_nested_scores[i],
          "svc", svc_clf.best_params_, svc_non_nested_scores[i],
          "lgbm", lgbm_clf.best_params_, lgbm_non_nested_scores[i]
         )
    
    # Nested CV with parameter optimization
    dt_nested_score = cross_val_score(estimator=dt_clf, X=X_train_std_pca, y=Y_train, cv=outer_cv)
    dt_nested_scores[i] = dt_nested_score.mean()
    knn_nested_score = cross_val_score(estimator=knn_clf, X=X_train_std_pca, y=Y_train, cv=outer_cv)
    knn_nested_scores[i] = knn_nested_score.mean()
    lr_nested_score = cross_val_score(estimator=lr_clf, X=X_train_std_pca, y=Y_train, cv=outer_cv)
    lr_nested_scores[i] = lr_nested_score.mean()
    svc_nested_score = cross_val_score(estimator=svc_clf, X=X_train_std_pca, y=Y_train, cv=outer_cv)
    svc_nested_scores[i] = svc_nested_score.mean()
    lgbm_nested_score = cross_val_score(estimator=lgbm_clf, X=X_train_std_pca, y=Y_train, cv=outer_cv)
    lgbm_nested_scores[i] = lgbm_nested_score.mean()
print(dt_nested_scores,
      knn_nested_scores,
      lr_nested_scores,
      svc_nested_scores,
      lgbm_nested_scores
     )







  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

Trial 0 dt {'max_depth': 51, 'criterion': 'gini'} 0.12350550549656598 knn {'weights': 'uniform', 'n_neighbors': 11} 0.0019782393669634025 lr {'C': 0.1} 0.0020964360587002098 svc {'kernel': 'rbf', 'gamma': 0.001, 'C': 1000} 0.04184853672152145 lgbm {'scale_pos_weight': 10, 'num_leaves': 100, 'n_estimators': 200, 'min_child_samples': 350, 'boosting_type': 'gbdt'} 0.26445593818538193








 20%|████████████████▌                                                                  | 1/5 [05:55<23:41, 355.32s/it]

Trial 1 dt {'max_depth': 51, 'criterion': 'entropy'} 0.1315284485411586 knn {'weights': 'distance', 'n_neighbors': 51} 0.0 lr {'C': 0.1} 0.0009551098376313276 svc {'kernel': 'rbf', 'gamma': 0.001, 'C': 1000} 0.03860616186843515 lgbm {'scale_pos_weight': 7, 'num_leaves': 250, 'n_estimators': 100, 'min_child_samples': 350, 'boosting_type': 'gbdt'} 0.23124332557229962








 40%|█████████████████████████████████▏                                                 | 2/5 [13:46<19:29, 389.95s/it]

...

Trial 3 dt {'max_depth': 41, 'criterion': 'gini'} 0.1251809657365656 knn {'weights': 'uniform', 'n_neighbors': 1} 0.08193829300015419 lr {'C': 0.1} 0.001952175975945076 svc {'kernel': 'rbf', 'gamma': 0.001, 'C': 1000} 0.03797321521025671 lgbm {'scale_pos_weight': 10, 'num_leaves': 400, 'n_estimators': 200, 'min_child_samples': 350, 'boosting_type': 'gbdt'} 0.2526644812853949








 80%|██████████████████████████████████████████████████████████████████▍                | 4/5 [23:46<05:45, 345.94s/it]

Trial 4 dt {'max_depth': 31, 'criterion': 'gini'} 0.13891705797176268 knn {'weights': 'distance', 'n_neighbors': 11} 0.0010482180293501049 lr {'C': 0.1} 0.0009980039920159682 svc {'kernel': 'rbf', 'gamma': 0.01, 'C': 1} 0.0 lgbm {'scale_pos_weight': 4, 'num_leaves': 100, 'n_estimators': 100, 'min_child_samples': 100, 'boosting_type': 'goss'} 0.09079424832543244








100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [32:06<00:00, 385.37s/it]

[0.13084303 0.12404006 0.11224371 0.13301094 0.12544558] [0.08858357 0.05443812 0.02848101 0.08193829 0.05744486] [0.00209644 0.00095511 0.0021097  0.00195218 0.000998  ] [0.06116116 0.07642565 0.04148375 0.04386118 0.04036001] [0.38959818 0.44244934 0.3594395  0.53011281 0.53657692]


In [49]:
# ncv - accuracy as metric
for i in tqdm(range(NUM_TRIALS)):
    inner_cv = KFold(n_splits=3, shuffle=True, random_state=i)
    outer_cv = KFold(n_splits=3, shuffle=True, random_state=i)
    scoring = 'accuracy'
    
    # Non_nested parameter search and scoring
    dt_clf = RandomizedSearchCV(estimator=dt, param_distributions=dt_grid, scoring=scoring, cv=inner_cv, n_iter=5)
    knn_clf = RandomizedSearchCV(estimator=knn, param_distributions=knn_grid, scoring=scoring, cv=inner_cv, n_iter=5)
    lr_clf = RandomizedSearchCV(estimator=lr, param_distributions=lr_grid, scoring=scoring, cv=inner_cv, n_iter=5)
    svc_clf = RandomizedSearchCV(estimator=svc, param_distributions=svc_grid, scoring=scoring, cv=inner_cv, n_iter=5)
    lgbm_clf = RandomizedSearchCV(estimator=lgbm, param_distributions=lgbm_grid, scoring=scoring, cv=inner_cv, n_iter=5)
    
    # Fit each model
    dt_clf.fit(X_train_std_pca, Y_train)
    knn_clf.fit(X_train_std_pca, Y_train)
    lr_clf.fit(X_train_std_pca, Y_train)
    svc_clf.fit(X_train_std_pca, Y_train)
    lgbm_clf.fit(X_train_std_pca, Y_train)
    
    dt_non_nested_scores[i] = dt_clf.best_score_
    knn_non_nested_scores[i] = knn_clf.best_score_
    lr_non_nested_scores[i] = lr_clf.best_score_
    svc_non_nested_scores[i] = svc_clf.best_score_
    lgbm_non_nested_scores[i] = lgbm_clf.best_score_
    
    print("Trial", i,
          "dt", dt_clf.best_params_, dt_non_nested_scores[i],
          "knn", knn_clf.best_params_, knn_non_nested_scores[i],
          "lr", lr_clf.best_params_, lr_non_nested_scores[i],
          "svc", svc_clf.best_params_, svc_non_nested_scores[i],
          "lgbm", lgbm_clf.best_params_, lgbm_non_nested_scores[i]
         )
    
    # Nested CV with parameter optimization
    dt_nested_score = cross_val_score(estimator=dt_clf, X=X_train_std_pca, y=Y_train, cv=outer_cv)
    dt_nested_scores[i] = dt_nested_score.mean()
    knn_nested_score = cross_val_score(estimator=knn_clf, X=X_train_std_pca, y=Y_train, cv=outer_cv)
    knn_nested_scores[i] = knn_nested_score.mean()
    lr_nested_score = cross_val_score(estimator=lr_clf, X=X_train_std_pca, y=Y_train, cv=outer_cv)
    lr_nested_scores[i] = lr_nested_score.mean()
    svc_nested_score = cross_val_score(estimator=svc_clf, X=X_train_std_pca, y=Y_train, cv=outer_cv)
    svc_nested_scores[i] = svc_nested_score.mean()
    lgbm_nested_score = cross_val_score(estimator=lgbm_clf, X=X_train_std_pca, y=Y_train, cv=outer_cv)
    lgbm_nested_scores[i] = lgbm_nested_score.mean()
print(dt_nested_scores,
      knn_nested_scores,
      lr_nested_scores,
      svc_nested_scores,
      lgbm_nested_scores
     )







  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

Trial 0 dt {'max_depth': 1, 'criterion': 'gini'} 0.9202439024390244 knn {'weights': 'uniform', 'n_neighbors': 51} 0.9202439024390244 lr {'C': 0.1} 0.9196747967479676 svc {'kernel': 'rbf', 'gamma': 0.01, 'C': 0.1} 0.9202439024390244 lgbm {'scale_pos_weight': 1, 'num_leaves': 250, 'n_estimators': 100, 'min_child_samples': 350, 'boosting_type': 'dart'} 0.9202439024390244








 20%|████████████████▌                                                                  | 1/5 [04:38<18:33, 278.49s/it]

Trial 1 dt {'max_depth': 21, 'criterion': 'gini'} 0.8524390243902439 knn {'weights': 'uniform', 'n_neighbors': 51} 0.9202439024390244 lr {'C': 0.1} 0.9199186991869919 svc {'kernel': 'rbf', 'gamma': 0.001, 'C': 1} 0.9202439024390244 lgbm {'scale_pos_weight': 1, 'num_leaves': 250, 'n_estimators': 500, 'min_child_samples': 600, 'boosting_type': 'gbdt'} 0.9182926829268293








 40%|█████████████████████████████████▏                                                 | 2/5 [10:29<15:00, 300.21s/it]

...

Trial 3 dt {'max_depth': 1, 'criterion': 'gini'} 0.9202439024390244 knn {'weights': 'uniform', 'n_neighbors': 51} 0.9202439024390244 lr {'C': 0.1} 0.9198373983739838 svc {'kernel': 'rbf', 'gamma': 0.001, 'C': 1} 0.9202439024390244 lgbm {'scale_pos_weight': 1, 'num_leaves': 200, 'n_estimators': 500, 'min_child_samples': 850, 'boosting_type': 'goss'} 0.9202439024390244








 80%|██████████████████████████████████████████████████████████████████▍                | 4/5 [20:36<05:07, 307.18s/it]

Trial 4 dt {'max_depth': 1, 'criterion': 'gini'} 0.9202439024390244 knn {'weights': 'distance', 'n_neighbors': 51} 0.9202439024390244 lr {'C': 0.1} 0.9198373983739838 svc {'kernel': 'rbf', 'gamma': 0.1, 'C': 0.1} 0.9202439024390244 lgbm {'scale_pos_weight': 1, 'num_leaves': 50, 'n_estimators': 100, 'min_child_samples': 350, 'boosting_type': 'gbdt'} 0.9200813008130081








100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [24:37<00:00, 295.44s/it]

[0.90211382 0.9202439  0.9202439  0.90219512 0.9202439 ] [0.9202439 0.9202439 0.9202439 0.9201626 0.9202439] [0.9196748 0.9199187 0.92      0.9198374 0.9198374] [0.9202439 0.9202439 0.9202439 0.9201626 0.9202439] [0.85471545 0.91186992 0.9195122  0.90918699 0.92      ]
